In [1]:
import base64
import requests
import time
import pandas as pd
import os

# OpenAI API Key - Replace with your GPT API key
api_key = ""

# Function to encode the image
def encode_image(image_path):
    with open(image_path, "rb") as image_file:
        return base64.b64encode(image_file.read()).decode('utf-8')

# Read CSV
questions_df = pd.read_csv("C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/VLAT Questions.csv")
questions = questions_df.to_dict('records')
responses_data = []


for i, question in enumerate(questions, start=1):
    time.sleep(5)
    try:
        print(f"\nProcessing question {i}:")
        print(question)
        
        # Path to your image
        image_path = "C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/" + str(question.get('vis', '')) + ".png"
        
        # Question text
        question_text = question.get('question: ', '')
        question_options = question.get('option:', '')
        correct_ans = str(question.get('correct', '')).strip()
        
        print(f"Processing image: {image_path}")
        print(f"Question: {question_text}")
        print(f"Options: {question_options}")
        print(f"Correct answer: {correct_ans}")
        
        # Getting the base64 string
        base64_image = encode_image(image_path)
        
        headers = {
            "Content-Type": "application/json",
            "Authorization": f"Bearer {api_key}"
        }
        
        payload = {
            "model": "gpt-4.5-preview-2025-02-27", # Use GPT-4o for vision capabilities
            "max_tokens": 5000,
            "temperature": 0.0,
            "messages": [
                {
                    "role": "system",
                    "content": "You are an assistant, skilled in reading and interpreting visually represented data."
                },
                {
                    "role": "user",
                    "content": [
                        {
                            "type": "text",
                            "text": 'I am about to show you a graph and ask you a multiple-choice question about that graph. \n\n' +
                                   'Task 1: Data Extraction and Table Creation: First, explicitly list ALL numerical values you can identify on both axes, then create a structured table using markdown syntax that includes ALL data points you identified above with appropriate column headers with units. \n \n' +
                                   'Task 2: Sort the data: Sort the data in descending order by the numerical values. \n \n' +
                                   'Task 3: Data Verification and Error Handling: Double-check if your table matches ALL elements in the graph by comparing each value in your table with the graph and updating your table with correct values, verify the sorting is correct, and before proceeding, confirm all corrections have been made and use ONLY the corrected data for analysis. \n \n' +
                                   'Task 4: Question Analysis: Using ONLY the verified data in your table, compare EACH value individually with the reference value, for "less than" comparisons mark ALL values that are even slightly below the reference, for "greater than" comparisons mark ALL values that are even slightly above the reference, and show each comparison on a new line. \n \n' +
                                   'Provide your reasoning with specific references to table values. \n\n' +
                                   'End with: "Correct Answer: ". Just write the value, nothing else. Do not write anything after this. \n\n' +
                                   'Let\'s solve this step by step.' +
                                   '\n\n' + question_text + " " + question_options
                        },
                        {
                            "type": "image_url",
                            "image_url": {
                                "url": f"data:image/png;base64,{base64_image}"
                            }
                        }
                    ]
                }
            ]
        }
        
        # Add retry logic for overloaded errors
        max_retries = 3
        retry_delay = 20  # seconds
        
        for retry in range(max_retries):
            try:
                time_start = time.perf_counter()
                response_raw = requests.post(
                    "https://api.openai.com/v1/chat/completions",
                    headers=headers,
                    json=payload
                )
                time_end = time.perf_counter()
                
                response = response_raw.json()
                
                # If we get a rate limit error, wait and retry
                if 'error' in response and response['error'].get('code') == 'rate_limit_exceeded':
                    if retry < max_retries - 1:  # Don't sleep on the last retry
                        print(f"\nAPI rate limited, waiting {retry_delay} seconds before retry {retry + 1}/{max_retries}")
                        time.sleep(retry_delay)
                        continue
                
                break  # If we get here, we either got a good response or a different error
                
            except Exception as e:
                print(f"Error during API call (attempt {retry + 1}/{max_retries}):", str(e))
                if retry < max_retries - 1:
                    time.sleep(retry_delay)
                    continue
                raise  # Re-raise the last exception if we've exhausted all retries
        print("\nAPI Response:", response)  # Debug print
        
        if 'error' in response:
            gpt_answer = f"Error: {response['error'].get('message', 'Unknown error')}"
            is_correct = "N/A"
        else:
            # Extract the answer after "Correct Answer: "
            full_response = response["choices"][0]["message"]["content"].strip()
            if "Correct Answer: " in full_response:
                gpt_answer = full_response.split("Correct Answer: ")[-1].strip()
                # Case-insensitive comparison after stripping whitespace
                is_correct = gpt_answer.strip().upper() == correct_ans.strip().upper()
            else:
                gpt_answer = "Error: No answer in correct format"
                is_correct = "N/A"
        
        responses_data.append([gpt_answer, time_end-time_start, is_correct])
        print(f"\nAnswer: {gpt_answer}")
        print(f"Time taken: {time_end-time_start:.2f} seconds")
        print(f"Correct? {is_correct}")
        
        # Increase delay between requests to 15 seconds
        time.sleep(max(15 - (time_end-time_start), 0))
        
    except Exception as e:
        print(f"Error processing question {i}:", str(e))
        responses_data.append([f"Error: {str(e)}", 0, "N/A"])

# Create Results directory if it doesn't exist
results_dir = "C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Results/"
os.makedirs(results_dir, exist_ok=True)

results_df = pd.DataFrame(responses_data, columns=['response', 'time', 'correct_bool'])
results_df.index = range(1, results_df.shape[0] + 1)
results_df.to_csv(results_dir + "GPT_VLAT_" + str(int(time.time())) + ".csv", index_label="id")
print("\n*** Finished ***")


Processing question 1:
{'id': 1, 'dropped': 'no', 'vis': 'VLAT_a', 'item': 'a_1', 'question: ': 'What was the price of a barrel of oil in February 2015? ', 'option:': '$57.36; $47.82; $50.24; $39.72', 'correct': '$50.24 '}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_a.png
Question: What was the price of a barrel of oil in February 2015? 
Options: $57.36; $47.82; $50.24; $39.72
Correct answer: $50.24

API Response: {'id': 'chatcmpl-B7WhmS1u2pMwtidPbwLdp0WlR6SxQ', 'object': 'chat.completion', 'created': 1741133714, 'model': 'gpt-4.5-preview-2025-02-27', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Task 1: Data Extraction and Table Creation\n\nNumerical values identified on axes:\n\nY-axis (Oil Price in $ per barrel):\n65, 60, 55, 50, 45, 40, 35\n\nX-axis (Months):\nJanuary, February, March, April, May, June, July, August, September, October, November, December\n\nExtracted Data Points (approximate values from the graph):\n\n


Processing question 4:
{'id': 4, 'dropped': 'no', 'vis': 'VLAT_a', 'item': 'a_4', 'question: ': 'Over the course of the second half of 2015, the price of a barrel of oil was ____________.', 'option:': 'rising; falling; staying', 'correct': 'falling'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_a.png
Question: Over the course of the second half of 2015, the price of a barrel of oil was ____________.
Options: rising; falling; staying
Correct answer: falling

API Response: {'id': 'chatcmpl-B7WlGTfrg8pz6CziQ2Si2Sg3tf6hL', 'object': 'chat.completion', 'created': 1741133930, 'model': 'gpt-4.5-preview-2025-02-27', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Task 1: Data Extraction and Table Creation\n\nNumerical values identified on axes:\n\nY-axis (Oil Price in $ per barrel):\n- 35, 40, 45, 50, 55, 60, 65\n\nX-axis (Months of 2015):\n- January, February, March, April, May, June, July, August, September, October, November, Decem


API Response: {'id': 'chatcmpl-B7WnG9XarrUyoAZlORns0OCzMCTks', 'object': 'chat.completion', 'created': 1741134054, 'model': 'gpt-4.5-preview-2025-02-27', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Task 1: Data Extraction and Table Creation\n\nNumerical values on axes:\n- Y-axis (Speed in Mbps): 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22\n- X-axis (Countries): Australia, China, Hong Kong, India, Indonesia, Japan, Malaysia, New Zealand, Singapore, South Korea, Sri Lanka, Taiwan, Thailand, Vietnam\n\nStructured Table (unsorted):\n\n| Country      | Speed (Mbps) |\n|--------------|--------------|\n| Australia    | 8            |\n| China        | 3            |\n| Hong Kong    | 16           |\n| India        | 2            |\n| Indonesia    | 2            |\n| Japan        | 15           |\n| Malaysia     | 4            |\n| New Zealand  | 9            |\n| Singapore    | 12           |\n| South Korea  | 20           |\n| Sri Lanka    | 5            |\n| Taiwan 


Processing question 9:
{'id': 9, 'dropped': 'no', 'vis': 'VLAT_b', 'item': 'b_4', 'question: ': 'How many countries in Asia is the average internet speed slower than Thailand?', 'option:': '5 countries; 6 countries; 7 countries; 8 countries', 'correct': '7 countries'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_b.png
Question: How many countries in Asia is the average internet speed slower than Thailand?
Options: 5 countries; 6 countries; 7 countries; 8 countries
Correct answer: 7 countries

API Response: {'id': 'chatcmpl-B7WrV02iyK7yPe1eN0vqd0717dzNX', 'object': 'chat.completion', 'created': 1741134317, 'model': 'gpt-4.5-preview-2025-02-27', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': "Task 1: Data Extraction and Table Creation\n\nNumerical values on axes:\n- Y-axis (Speed in Mbps): 0, 2, 4, 6, 8, 10, 12, 14, 16, 18, 20, 22\n- X-axis (Countries): Australia, China, Hong Kong, India, Indonesia, Japan, Malaysia, New Zealand,


Processing question 12:
{'id': 12, 'dropped': 'no', 'vis': 'VLAT_c', 'item': 'c_3', 'question: ': 'In which city is the cost of soda the highest?', 'option:': 'New York City; Las Vegas; Atalanta; Washington D.C.', 'correct': 'Washington D.C.'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_c.png
Question: In which city is the cost of soda the highest?
Options: New York City; Las Vegas; Atalanta; Washington D.C.
Correct answer: Washington D.C.

API Response: {'id': 'chatcmpl-B7WvEdBd50JhD3QEjTybCHLul7ZmQ', 'object': 'chat.completion', 'created': 1741134548, 'model': 'gpt-4.5-preview-2025-02-27', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Task 1: Data Extraction and Table Creation\n\nNumerical values on axes:\n- Y-axis (Cost in $): 0, 5, 10, 15, 20, 25, 30, 35, 40, 45, 50, 55, 60\n- X-axis (Cities): New York City, Las Vegas, Honolulu, Atlanta, Boston, Washington D.C., Chicago, Orlando, San Francisco, Seattle\n\nExtracted Soda

...


Processing question 50:
{'id': 50, 'dropped': 'yes', 'vis': 'VLAT_h', 'item': 'h_4', 'question: ': 'In general, the number of stations of the metro systems of the world is evenly distributed.', 'option:': 'True; False', 'correct': 'TRUE'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_h.png
Question: In general, the number of stations of the metro systems of the world is evenly distributed.
Options: True; False
Correct answer: TRUE

API Response: {'id': 'chatcmpl-B7Xrl4tGfbW529Pd7u2Zdrj2MC2Zl', 'object': 'chat.completion', 'created': 1741138177, 'model': 'gpt-4.5-preview-2025-02-27', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Task 1: Data Extraction and Table Creation\n\nNumerical values identified on axes:\n\nX-axis (Number of Stations):\n- 100, 150, 200, 250, 300, 350, 400, 450\n\nY-axis (Total System Length in km):\n- 150, 200, 250, 300, 350, 400, 450, 500, 550, 600\n\nData points extracted from the graph:\n\n| City     


Processing question 52:
{'id': 52, 'dropped': 'no', 'vis': 'VLAT_h', 'item': 'h_6', 'question: ': 'A group of the metro systems of the world has approximately 300 stations and around a 200 km system length.', 'option:': 'True; False', 'correct': 'FALSE'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_h.png
Question: A group of the metro systems of the world has approximately 300 stations and around a 200 km system length.
Options: True; False
Correct answer: FALSE

API Response: {'id': 'chatcmpl-B7XupEL2Ojz3GU003FZ49rNGThBdo', 'object': 'chat.completion', 'created': 1741138367, 'model': 'gpt-4.5-preview-2025-02-27', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Task 1: Data Extraction and Table Creation\n\nNumerical values identified on axes:\n\nX-axis (Number of Stations):\n- 100, 150, 200, 250, 300, 350, 400, 450\n\nY-axis (Total System Length in km):\n- 150, 200, 250, 300, 350, 400, 450, 500, 550, 600\n\nExtracted Data Poin


Processing question 54:
{'id': 54, 'dropped': 'no', 'vis': 'VLAT_h', 'item': 'h_8', 'question: ': 'The metro system in Shanghai has more ridership than the metro system in Beijing.', 'option:': 'True; False', 'correct': 'FALSE'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_h.png
Question: The metro system in Shanghai has more ridership than the metro system in Beijing.
Options: True; False
Correct answer: FALSE

API Response: {'id': 'chatcmpl-B7XxyeMQhIIv455tcJLJPREWzw83x', 'object': 'chat.completion', 'created': 1741138562, 'model': 'gpt-4.5-preview-2025-02-27', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Task 1: Data Extraction and Table Creation\n\nNumerical values identified on axes:\n\nX-axis (Number of Stations):\n100, 150, 200, 250, 300, 350, 400, 450\n\nY-axis (Total System Length in km):\n150, 200, 250, 300, 350, 400, 450, 500, 550, 600\n\nRidership (bn per year) bubble sizes:\n1.5, 2.5, 3.5\n\nExtracted Data Poin


Processing question 57:
{'id': 57, 'dropped': 'no', 'vis': 'VLAT_k', 'item': 'k_3', 'question: ': 'In 2015, the unemployment rate for Washington (WA) was higher than that of Wisconsin (WI).', 'option:': 'True; False', 'correct': 'TRUE'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_k.png
Question: In 2015, the unemployment rate for Washington (WA) was higher than that of Wisconsin (WI).
Options: True; False
Correct answer: TRUE

API Response: {'id': 'chatcmpl-B7Y0d891yPlsefbimYLilOzMQGXTy', 'object': 'chat.completion', 'created': 1741138727, 'model': 'gpt-4.5-preview-2025-02-27', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'Task 1: Data Extraction and Table Creation\n\nNumerical values identified on the color scale (unemployment rate %):\n0.0%, 1.1%, 2.3%, 3.4%, 4.6%, 5.7%, 6.9%\n\nExtracted data for the states in question:\n- Washington (WA): approximately 5.7%\n- Wisconsin (WI): approximately 4.6%\n\n| State | Unemployment


Processing question 60:
{'id': 60, 'dropped': 'no', 'vis': 'VLAT_l', 'item': 'l_3', 'question: ': 'The number of unique visitors for Amazon was more than that of Yahoo in 2010.', 'option:': 'True; False', 'correct': 'FALSE'}
Processing image: C:/Users/amitc/OneDrive/Desktop/New folder (7)/VLAT/Images/VLAT_l.png
Question: The number of unique visitors for Amazon was more than that of Yahoo in 2010.
Options: True; False
Correct answer: FALSE

API Response: {'id': 'chatcmpl-B7Y3TzOo6Nb5acuSGGxNoqdlMHIx3', 'object': 'chat.completion', 'created': 1741138903, 'model': 'gpt-4.5-preview-2025-02-27', 'choices': [{'index': 0, 'message': {'role': 'assistant', 'content': 'The graph provided does not contain any numerical values on either axis. It is a treemap visualization that shows relative sizes of categories and websites based on the number of unique visitors, but it does not explicitly provide numerical values or units for these visitors.\n\nTask 1: Data Extraction and Table Creation:\nSince